In [45]:
import requests
from bs4 import BeautifulSoup
import json

In [46]:
# URL of the 1st article
url = "https://www.goodreads.com/book/show/22034.The_Godfather"

In [47]:
# Fetch the page
response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'html.parser')

In [48]:
print(soup)

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8" data-next-head=""/><meta content="width=device-width" data-next-head="" name="viewport"/><link href="/favicon.ico" rel="icon"/><link href="https://www.goodreads.com/book/show/22034.The_Godfather" rel="canonical"/><link href="https://www.goodreads.com/en/book/show/22034.The_Godfather" hreflang="en" rel="alternate"/><link href="https://www.goodreads.com/it/book/show/22034.The_Godfather" hreflang="it" rel="alternate"/><link href="https://www.goodreads.com/es/book/show/22034.The_Godfather" hreflang="es" rel="alternate"/><link href="https://www.goodreads.com/zh/book/show/22034.The_Godfather" hreflang="zh" rel="alternate"/><link href="https://www.goodreads.com/id/book/show/22034.The_Godfather" hreflang="id" rel="alternate"/><link href="https://www.goodreads.com/fa/book/show/22034.The_Godfather" hreflang="fa" rel="alternate"/><link href="https://www.goodreads.com/ar/book/show/22034.The_Godfather" hreflang="ar" rel="alternate"/><link 

In [51]:
reviewCards = soup.find_all('article', class_='ReviewCard')
print(f"Total number of ReviewCard: {len(reviewCards)}")

Total number of ReviewCard: 30


In [ ]:
print(reviewCards[0])
#name: ReviewerProfile__name
#text: ReviewText__content
#rating: ShelfStatus


<article aria-label="Review by Diane Wallace" class="ReviewCard"><div class="ReviewCard__profile"><div class="ReviewerProfile ReviewerProfile--medium"><section class="ReviewerProfile__avatar"><a class="Avatar Avatar--medium" href="https://www.goodreads.com/user/show/43342511-diane-wallace" tabindex="0"><img alt="Profile Image for Diane Wallace." class="Avatar__image" data-nimg="1" data-testid="image" decoding="async" height="56" loading="lazy" src="/_next/image?url=https%3A%2F%2Fi.gr-assets.com%2Fimages%2FS%2Fcompressed.photo.goodreads.com%2Fusers%2F1621272868i%2F43342511._UY200_CR62%2C0%2C200%2C200_.jpg&amp;w=128&amp;q=75" srcset="/_next/image?url=https%3A%2F%2Fi.gr-assets.com%2Fimages%2FS%2Fcompressed.photo.goodreads.com%2Fusers%2F1621272868i%2F43342511._UY200_CR62%2C0%2C200%2C200_.jpg&amp;w=64&amp;q=75 1x, /_next/image?url=https%3A%2F%2Fi.gr-assets.com%2Fimages%2FS%2Fcompressed.photo.goodreads.com%2Fusers%2F1621272868i%2F43342511._UY200_CR62%2C0%2C200%2C200_.jpg&amp;w=128&amp;q=75 2

In [53]:
reviews = []
for card in reviewCards:
    # Extract reviewer name
    name_tag = card.find(class_='ReviewerProfile__name')
    name = name_tag.get_text(strip=True) if name_tag else None

    # Extract review text
    text_tag = card.find(class_='ReviewText__content')
    text = text_tag.get_text(strip=True) if text_tag else None

    # Extract rating
    rating_tag = card.find(class_='ShelfStatus')
    rating = rating_tag.get_text(strip=True) if rating_tag else None

    reviews.append({
        "name": name,
        "text": text,
        "rating": rating
    })

print(reviews)

[{'name': 'Diane Wallace', 'text': 'The Best! a treasure of a story,plot and movie to keep forever...good storytelling and amazing writing (paperback!)', 'rating': ''}, {'name': 'Brina', 'text': "Mario Puzo wrote The Godfather, a book that was to become an instant classic, at a time when both his personal life and that of the United States were in transition. Puzo's first two novels had not sold well, and he was almost penniless. A young, up and coming author, Puzo sought to write his version of the great American novel that would also work well on the silver screen. Meanwhile, in a time of war, the nation was beginning to view the mafia rather than Cowboys as the great American hero. According to afterward author Peter Bart, America's perception of the mafia as positive citizens is what allowed Puzo to thrust The Godfather into the forefront of American society.As someone who has never experienced the award winning film starring Marlon Brando and Al Pacino, Don Corleone and the world 

In [54]:
import csv
from datetime import datetime
import os

# Create output directory if it doesn't exist
output_dir = os.path.join(os.getcwd(), "output")
os.makedirs(output_dir, exist_ok=True)

# Generate timestamped filename
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_path = os.path.join(output_dir, f"godfather_reviews_{timestamp}.csv")

with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['name', 'text', 'rating']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for review in reviews:
        writer.writerow(review)

print(f"Saved reviews to {csv_path}")

Saved reviews to /Users/amtea/Documents/GitHub/msc-5481-data-engineering/Assignment1/output/godfather_reviews_20250929_210742.csv


In [30]:
# Initialize a list to store the reviews
reviews = []

# Find the review elements
review_elements = soup.find_all('div', class_='review')

for review in review_elements:
    # Extract review text
    review_text = review.find('span', class_='readable').get_text(strip=True)
    
    # Extract user name
    user_name = review.find('a', class_='user').get_text(strip=True)
    
    # Extract date
    date = review.find('a', class_='reviewDate').get_text(strip=True)
    
    # Extract rating
    rating_element = review.find('span', class_='staticStars')
    rating = int(rating_element['title'].split()[0]) if rating_element else None
    
    # Add the extracted data to the reviews list
    reviews.append({
        "review_text": review_text,
        "user": user_name,
        "date": date,
        "rating": rating,
        "reply": None  # Assuming no replies are present in this case
    })

# Save the data to a JSON file
with open('the_godfather_reviews.json', 'w', encoding='utf-8') as f:
    json.dump(reviews, f, ensure_ascii=False, indent=4)

print(f"Scraped {len(reviews)} reviews.")

Scraped 0 reviews.
